In [1]:
import numpy as np
import gensim
import string

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

In [2]:
path = 'data/trump_sequences.txt'

print('\nPreparing the sentences...')
max_sentence_len = 40
with open(path, encoding='utf8') as file_:
    docs = file_.readlines()
#sentences = [[word for word in doc.lower().translate(None, string.punctuation).split()[:max_sentence_len]] for doc in docs]
sentences = [x.strip() for x in docs] 
print('Num sentences:', len(sentences))


Preparing the sentences...
Num sentences: 164521


In [3]:
sentences = [[word for word in document.split()] for document in sentences]
sentences

[['great',
  'to',
  'be',
  'with',
  'our',
  'wonderful',
  'men',
  'and',
  'women',
  'of',
  'the',
  'what',
  'a',
  'job',
  'they',
  'are',
  'doing',
  'the',
  'problem',
  'is',
  'not',
  'the',
  'very',
  'talented',
  'lowflying',
  'helicopter',
  'pilots',
  'wanting',
  'to',
  'save',
  'our',
  'city',
  'the',
  'problem',
  'is',
  'the',
  'arsonists',
  'looters',
  'criminals',
  'and',
  'anarchists',
  'wanting',
  'to',
  'destroy',
  'it',
  'and',
  'our',
  'country',
  'michael',
  'is',
  'tough'],
 ['to',
  'be',
  'with',
  'our',
  'wonderful',
  'men',
  'and',
  'women',
  'of',
  'the',
  'what',
  'a',
  'job',
  'they',
  'are',
  'doing',
  'the',
  'problem',
  'is',
  'not',
  'the',
  'very',
  'talented',
  'lowflying',
  'helicopter',
  'pilots',
  'wanting',
  'to',
  'save',
  'our',
  'city',
  'the',
  'problem',
  'is',
  'the',
  'arsonists',
  'looters',
  'criminals',
  'and',
  'anarchists',
  'wanting',
  'to',
  'destroy',
 

In [4]:
word_model = gensim.models.Word2Vec(sentences, size=200, min_count=1, window=5, iter=100)

pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape

print(pretrained_weights)
print(vocab_size)
print(emdedding_size)

[[-1.2261695e+00  1.0299642e+00 -7.4431992e-01 ... -3.8074005e-01
   6.6487592e-01  3.7398872e+00]
 [-8.0827928e-01 -1.5084667e+00  3.2292944e-03 ... -5.1086783e-01
   4.4958243e-01  4.9683604e+00]
 [ 6.1321717e-02 -2.8686409e+00 -8.0607778e-01 ... -2.5411437e+00
  -1.7407489e+00  4.8168573e+00]
 ...
 [ 1.2364806e-02  1.7643697e+00  2.6639433e+00 ... -2.4473634e+00
  -6.5253276e-01 -4.9953112e-01]
 [-1.4483275e+00 -1.0640080e+00 -3.5199240e-02 ...  2.4835556e+00
  -4.7765931e-01 -3.5277164e-01]
 [-2.2837336e+00 -1.1876338e-01  5.0471473e-01 ...  2.6826808e+00
  -2.4375188e-01  4.9326995e-01]]
10304
200


Z:\Programacion\miniconda3\envs\deepl\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
print('\nTraining LSTM...')
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[pretrained_weights], trainable=False),
    LSTM(emdedding_size, dropout=0.2, return_sequences=True),
    LSTM(emdedding_size, dropout=0.2),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


Training LSTM...


In [26]:
def word2idx(word):
    return word_model.wv.vocab[word].index
def idx2word(idx):
    return word_model.wv.index2word[idx]


train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence[:-1]):
        if t < 40:
            train_x[i, t] = word2idx(word)
    train_y[i] = word2idx(sentence[-1])

In [27]:
model.fit(train_x, train_y, batch_size=128, epochs=20, validation_split=0.1)
model.save('models/trump_model.h5')

Train on 148068 samples, validate on 16453 samples
Epoch 1/20
148068/148068 [==============================] - 21s 142us/sample - loss: 6.9471 - val_loss: 7.0243
Epoch 2/20
148068/148068 [==============================] - 18s 123us/sample - loss: 6.7781 - val_loss: 7.0609
Epoch 3/20
148068/148068 [==============================] - 18s 123us/sample - loss: 6.6847 - val_loss: 7.0242
Epoch 4/20
148068/148068 [==============================] - 18s 123us/sample - loss: 6.6036 - val_loss: 7.0493
Epoch 5/20
148068/148068 [==============================] - 18s 123us/sample - loss: 6.5319 - val_loss: 7.0880
Epoch 6/20
148068/148068 [==============================] - 19s 129us/sample - loss: 6.4547 - val_loss: 7.1283
Epoch 7/20
148068/148068 [==============================] - 18s 121us/sample - loss: 6.3730 - val_loss: 7.1704
Epoch 8/20
148068/148068 [==============================] - 19s 128us/sample - loss: 6.2811 - val_loss: 7.2368
Epoch 9/20
148068/148068 [==============================] - 1